Importamos las librerías necesarias.

In [157]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Cargamos las tablas de la base de datos en un DataFrame de pandas.

In [170]:
# Conectarse a la base de datos SQLite
conexion = sqlite3.connect('bookmaker.db')

# Leer las tablas en DataFrames de pandas
df_apuestas = pd.read_sql_query("SELECT * FROM apuestas", conexion)
df_partidos = pd.read_sql_query("SELECT * FROM partidos", conexion)
df_equipos = pd.read_sql_query("SELECT * FROM equipos", conexion)
df_empresa = pd.read_sql_query("SELECT * FROM empresa", conexion)
df_clientes = pd.read_sql_query("SELECT * FROM clientes", conexion)
df_cuotas = pd.read_sql_query("SELECT * FROM cuotas", conexion)

# Cerrar la conexión
conexion.close()

Vemos las 5 primeras filas de cada tabla para entender su estructura.

In [171]:
df_apuestas.head()

,id,fecha,monto,equipo_ganador_id,partido,cliente,ganacia
0,1,2021-12-15 01:08:30.211674,300.0,2.0,1,88,891.0
1,2,2021-12-15 01:08:30.211674,300.0,29.0,1,88,891.0
2,3,2021-12-15 01:35:34.965516,300.0,2.0,1,88,891.0
3,4,2021-12-15 01:35:34.965516,300.0,29.0,1,88,891.0
4,5,2021-12-15 01:35:34.965516,300.0,2.0,1,88,891.0


In [172]:
df_partidos.head()

,id,fecha,equipo_local,equipo_visistante,finalizado,ganador
0,1,2021-12-15 00:51:23.181740,29,2,0,None
1,2,2021-12-15 00:51:23.181740,31,19,0,None
2,3,2021-12-15 00:51:23.181740,25,24,0,None
3,4,2021-12-15 00:51:23.181740,11,6,0,None
4,5,2021-12-15 00:51:23.181740,4,5,0,None


In [173]:
df_equipos.head()

,id,nombre,escudo,pais,puntaje,activado
0,1,Ajax,ajax.png,Paises Bajos,14.0,1
1,2,Atalanta,atalanta.png,Italia,6.0,1
2,3,Atlético,atletico.png,España,11.0,1
3,4,Barcelona,barcelona.png,España,17.0,1
4,5,Bayern,bayern.png,Alemania,16.0,1


In [174]:
df_empresa.head()

,id,razon_social,email,activado


In [175]:
df_clientes.head()

,id,nombre,apellido,email,activado
0,1,Lope,Perales,pascualperera@vila.es,1
1,2,Isaura,Canet,gmerino@rocamora.es,1
2,3,Olga,Chaves,amayaprats@llado.com,1
3,4,Celestino,Hoyos,chedelgado@moraleda-pintor.com,1
4,5,Miguel,Milla,inigoprat@gmail.com,1


In [176]:
df_cuotas.head()

,id,local,empate,visitante,partido_id
0,1,1.40,2.97,5.80,1
1,2,5.73,5.08,3.86,2
2,3,5.45,5.99,5.34,3
3,4,1.95,5.08,4.78,4
4,5,4.44,5.99,4.61,5


A continuación vamos a utilizar la tabla cuotas para predecir el resultado de la tabla partidos. De forma que daremos el resultado cuya cuota sea la más baja en cada partido.

Antes que nada, vamos a limpiar los datos. En primer lugar, vamos a eliminar las filas que tengan valores nulos en las columnas en las que no deberian tenerlos. 

In [177]:
# Eliminamos las filas con valores nulos del DataFrame de cuotas
df_cuotas.dropna(inplace=True)

# Eliminamos las filas con valores nulos del DataFrame de partidos. La columna ganador tiene valores nulos siempre asi que en esa columna no comprobamos si hay valores nulos
df_partidos.dropna(inplace=True, subset=['id','fecha','equipo_local','equipo_visistante','finalizado'])

Otro problema que puede haber en estos dos dataframes es que haya partidos duplicados.

In [178]:
# Nos aseguramos que no haya partidos duplicados en el DataFrame de cuotas
df_cuotas.drop_duplicates(subset='partido_id', keep='first', inplace=True)

# Nos aseguramos que no haya partidos duplicados en el DataFrame de partidos
df_partidos.drop_duplicates(subset='id', keep='first', inplace=True)

In [179]:
# Encontrar el tipo de resultado con la cuota más baja (1, X, o 2)
df_cuotas['resultado'] = df_cuotas[['local', 'empate', 'visitante']].idxmin(axis=1)

# Crear un nuevo DataFrame con las columnas necesarias
df_relacion = df_cuotas[['partido_id', 'resultado']]

# Fusionar los DataFrames en función de la columna 'partido_id'
df_partidos = pd.merge(df_partidos, df_relacion, left_on='id', right_on='partido_id', how='left')

# Mapear el resultado a la columna 'ganador' en el DataFrame de partidos
mapeo_resultados = {'local': '1', 'empate': 'X', 'visitante': '2'}
df_partidos['ganador'] = df_partidos['resultado'].map(mapeo_resultados)

# Eliminar columnas adicionales
df_partidos.drop(['partido_id', 'resultado'], axis=1, inplace=True)
df_cuotas.drop(['resultado'], axis=1, inplace=True)

In [180]:
df_partidos.head(5)

,id,fecha,equipo_local,equipo_visistante,finalizado,ganador
0,1,2021-12-15 00:51:23.181740,29,2,0,1
1,2,2021-12-15 00:51:23.181740,31,19,0,2
2,3,2021-12-15 00:51:23.181740,25,24,0,2
3,4,2021-12-15 00:51:23.181740,11,6,0,1
4,5,2021-12-15 00:51:23.181740,4,5,0,1
